## Interim solution to address expenditure data issue for March 15 presentation

* Using data from Data Link.

In [1]:
import numpy as np
import pandas as pd
import TIRCP_functions
from siuba import *
from calitp import *

GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/tircp/"

pd.options.display.max_columns = 50
pd.options.display.max_rows = 120
pd.options.display.float_format = "{:.2f}".format

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
E0309 00:41:30.850898814    1574 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0309 00:41:31.365049418    1574 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


In [2]:
#Expenditure information from the Enterprise data Linda pulled.
enterprise = "Enterprise.xlsx"
enterprise_df = pd.read_excel(f"{GCS_FILE_PATH}{enterprise}")
enterprise_df = to_snakecase(enterprise_df)

In [3]:
enterprise_df.head(1)

,fy,cycle,fund,appr_catg,appr_unit,pec,project,project_name,project_short_name,phase,old_ea,capital_exp,local_exp,support_exp,other_exp,tot_exp
0,2016,One,3228,1415,15101,3010070,16000007,Refurbishment of Seven Light Rail Vehicles,64-R340GA,S,R340GA,0,2182302.44,0,0,2182302.44


In [4]:
enterprise_df = enterprise_df.rename(columns = {'project':'project_id'})

In [5]:
#keep only columns necessary.
enterprise_df = enterprise_df[['fy','cycle','project_id','project_name','tot_exp']]

In [6]:
enterprise_df.shape

(254, 5)

In [7]:
#check out data type
enterprise_df.dtypes

fy                int64
cycle            object
project_id        int64
project_name     object
tot_exp         float64
dtype: object

In [8]:
#aggregate so that each project name and each cycle has only one record for total expenses
enterprise_df_group = enterprise_df.groupby(['project_name','cycle']).agg({'tot_exp':'sum'}).reset_index()

In [9]:
enterprise_df_group.shape

(88, 3)

In [10]:
enterprise_df_group.head(1)

,project_name,cycle,tot_exp
0,#Electrify Anaheim (ZEV Bus Procurement),Three,13492703.78


In [11]:
enterprise_df_project_id = enterprise_df.groupby(['project_id','cycle']).agg({'tot_exp':'sum'}).reset_index()

In [12]:
enterprise_df_project_id.shape

(106, 3)

####  <font color='red'> 106 project ids given...we only have 74 projects in TIRCP?</font> 

In [13]:
enterprise_df_project_id.project_id.nunique()

106

### Merging in project sheet. Only 22/74 matches were "both" :( 

In [14]:
#tableau
project_df = TIRCP_functions.project()

In [15]:
project_df.shape

(74, 32)

In [16]:
joined_proj = pd.merge(project_df, enterprise_df_group, left_on='Project_Title', right_on='project_name', how = 'left', indicator = True)

In [17]:
joined_proj._merge.value_counts()

left_only     52
both          22
right_only     0
Name: _merge, dtype: int64

In [18]:
joined_proj.head(1)

,Award_Year,Project_#,Local_Agency,Vendor_ID_#,Project_Title,District,County,Key_Project_Elements,Master_Agreement_Number,Master_Agreement_Expiration_Date,Project_Manager,Regional_Coordinator,Technical_Assistance-CALTP_(Y/N),Technical_Assistance-Fleet_(Y/N),Technical_Assistance-Network_Integration_(Y/N),Technical_Assistance-Priority_Population_(Y/N),Total_Project_Cost,TIRCP_project_sheet,Allocated_Amount,Unallocated_amt_project_sheet,Percentge_Allocated,Expended_Amt_project_sheet,Other_Funds_Involved,Award_Cycle,Local_Agency_Address,Local_Agency_City,Local_Agency_Zip,Local_Agency_Contact,Local_Agency_Email,Local_Agency_Phone_Number,Comments/Additional_Contacts,PPNO,project_name,cycle,tot_exp,_merge
0,2015,1,Antelope Valley Transit Authority (AVTA),TBD,Regional Transit Interconnectivity & Environme...,7,LA,Purchase 13 60-foot articulated BRT buses and ...,64AVTA2015MA,2024-04-01,Yesenia,Ryan Greenway,NaN,NaN,NaN,NaN,39478000.00,24403000.00,24403000,0,1.00,21714177.53,0.00,1.00,"42210 6th Street West\nLancaster, CA 93534",Lancaster,93534,Judy Fry,Jfry@avta.com,(611) 729-2234,NaN,CP005,NaN,NaN,NaN,left_only


In [20]:
#make a subset of joined df to check out _merges.
joined_subset_proj = joined_proj[['PPNO','project_name','Project_Title','tot_exp','Expended_Amt_project_sheet', '_merge']] 

In [21]:
joined_subset_proj.loc[joined_subset_proj['_merge'] == 'both'] 

,PPNO,project_name,Project_Title,tot_exp,Expended_Amt_project_sheet,_merge
3,CP007,Pacific Surfliner Transit Transfer Program,Pacific Surfliner Transit Transfer Program,277840.47,277840.47,both
5,CP004,Bravo! Route 560 Rapid Buses,Bravo! Route 560 Rapid Buses,2320000.00,0.00,both
9,CP006,SFMTA Light Rail Vehicle Fleet Expansion,SFMTA Light Rail Vehicle Fleet Expansion,41181000.00,0.00,both
10,CP014,ACE Wayside Power Project,ACE Wayside Power Project,15000.00,200000.00,both
17,CP016,Metropolitan Rapid Transit and Rail Connectivi...,Metropolitan Rapid Transit and Rail Connectivi...,4083000.00,0.00,both
19,CP020,Metro Red Line and Purple Line Core Capacity I...,Metro Red Line and Purple Line Core Capacity I...,22828081.56,4220881.34,both
21,CP017,OC Streetcar and OCTA System-Wide Mobile Ticke...,OC Streetcar and OCTA System-Wide Mobile Ticke...,2413997.30,0.00,both
22,CP022,Peninsula Corridor Electrification Project,Peninsula Corridor Electrification Project,16100400.95,14102518.86,both
24,1230,Redlands Passenger Rail Project,Redlands Passenger Rail Project,9204000.00,9204000.00,both
25,CP006,Light Rail Modernization and Expansion Program,Light Rail Modernization and Expansion Program,45092000.00,0.00,both


In [22]:
joined_subset_proj.loc[joined_subset_proj['_merge'] == 'left_only'] 

,PPNO,project_name,Project_Title,tot_exp,Expended_Amt_project_sheet,_merge
0,CP005,NaN,Regional Transit Interconnectivity & Environme...,NaN,21714177.53,left_only
1,CP012,NaN,Travel Time Reduction Project,NaN,4619999.90,left_only
2,CP015,NaN,Willowbrook/Rosa Parks Station & Blue Line Lig...,NaN,38494000.00,left_only
4,CP013,NaN,Monterey Bay Operations and Maintenance Facili...,NaN,0.00,left_only
6,CP001,NaN,Sacramento Regional Transit's Refurbishment of...,NaN,0.00,left_only
7,CP003,NaN,South Bay Bus Rapid Transit,NaN,4000000.00,left_only
8,CP008,NaN,San Diego Metropolitan Transit System Trolley ...,NaN,28176000.00,left_only
11,CP011,NaN,Bus Rapid Transit – Martin Luther King Corrido...,NaN,6841000.00,left_only
12,CP002,NaN,Purchase of Nine Fuel-Efficient Tier IV EMD F-...,NaN,37583067.00,left_only
13,CP010,NaN,SMART Rail Car Capacity Project,NaN,1650000.00,left_only


In [23]:
joined_proj = joined_proj.drop(columns =['_merge'])

In [24]:
joined_proj.shape

(74, 35)

### Merge in with allocation to see if results are better
* How is there so many unique project id numbers? When there are only 74 projects...

In [25]:
#read in allocation df because that's the sheet with project ids. 
allocation = TIRCP_functions.allocation()

In [26]:
#keep only columns that are relevant.
allocation_df = allocation[['Award_Year','Project_ID','PPNO']].drop_duplicates(subset = ['Project_ID'])

In [27]:
allocation_df.shape

(172, 3)

In [28]:
allocation_df.Project_ID.nunique()

171

In [29]:
allocation_df.dtypes

Award_Year     int64
Project_ID    object
PPNO          object
dtype: object

In [30]:
enterprise_df_project_id.dtypes

project_id      int64
cycle          object
tot_exp       float64
dtype: object

In [31]:
enterprise_df_project_id.head(1)

,project_id,cycle,tot_exp
0,16000007,One,6315208.89


In [32]:
joined_alloc = pd.merge(allocation_df, enterprise_df_project_id, left_on='Project_ID', right_on='project_id', how = 'left', indicator = True)

In [33]:
joined_alloc._merge.value_counts()

both          104
left_only      68
right_only      0
Name: _merge, dtype: int64

In [34]:
#only keep "both" results...
joined_alloc2 = joined_alloc.loc[joined_alloc['_merge'] == 'both'].drop(columns =['_merge'])

In [35]:
#group by so that only one row for each PPNO number & Year
joined_alloc2 = (joined_alloc2
                  .groupby(['PPNO', 'cycle'])
                  .agg({'tot_exp':'sum'})
                  .reset_index()
                 )

In [36]:
joined_alloc2.shape

(51, 3)

In [37]:
#rename cols before joining with "joined" df 
joined_alloc2 = joined_alloc2.add_prefix('from_joined_alloc_df_')

In [38]:
joined_alloc2

,from_joined_alloc_df_PPNO,from_joined_alloc_df_cycle,from_joined_alloc_df_tot_exp
0,1155,Three,486509.70
1,1230,Two,9204000.00
2,2320B,Three,500000.00
3,CP001,One,6315208.89
4,CP002,One,38023039.68
5,CP003,One,4000000.00
6,CP004,One,2320000.00
7,CP005,One,22284205.53
8,CP006,One,41181000.00
9,CP006,Three,26867000.00


### Joining my two merged sheets together sheet together...

* Drop duplicates 

In [39]:
final_join = pd.merge(joined_proj, joined_alloc2, left_on = 'PPNO', right_on = 'from_joined_alloc_df_PPNO',how = 'left', indicator = True)

In [40]:
final_join.head(1)

,Award_Year,Project_#,Local_Agency,Vendor_ID_#,Project_Title,District,County,Key_Project_Elements,Master_Agreement_Number,Master_Agreement_Expiration_Date,Project_Manager,Regional_Coordinator,Technical_Assistance-CALTP_(Y/N),Technical_Assistance-Fleet_(Y/N),Technical_Assistance-Network_Integration_(Y/N),Technical_Assistance-Priority_Population_(Y/N),Total_Project_Cost,TIRCP_project_sheet,Allocated_Amount,Unallocated_amt_project_sheet,Percentge_Allocated,Expended_Amt_project_sheet,Other_Funds_Involved,Award_Cycle,Local_Agency_Address,Local_Agency_City,Local_Agency_Zip,Local_Agency_Contact,Local_Agency_Email,Local_Agency_Phone_Number,Comments/Additional_Contacts,PPNO,project_name,cycle,tot_exp,from_joined_alloc_df_PPNO,from_joined_alloc_df_cycle,from_joined_alloc_df_tot_exp,_merge
0,2015,1,Antelope Valley Transit Authority (AVTA),TBD,Regional Transit Interconnectivity & Environme...,7,LA,Purchase 13 60-foot articulated BRT buses and ...,64AVTA2015MA,2024-04-01,Yesenia,Ryan Greenway,NaN,NaN,NaN,NaN,39478000.00,24403000.00,24403000,0,1.00,21714177.53,0.00,1.00,"42210 6th Street West\nLancaster, CA 93534",Lancaster,93534,Judy Fry,Jfry@avta.com,(611) 729-2234,NaN,CP005,NaN,NaN,NaN,CP005,One,22284205.53,both


In [41]:
final_join = final_join.drop_duplicates(subset = ['PPNO', 'Project_Title','Award_Year'])

In [42]:
final_join.shape

(74, 39)

In [43]:
final_join._merge.value_counts()

both          49
left_only     25
right_only     0
Name: _merge, dtype: int64

In [47]:
final_join.PPNO.nunique()

70

In [48]:
project_df.PPNO.nunique()

70

#### Checking out our 3 columns of expended amounts, lol:
* from original project data sheet
* first merge of Data Link with Projects df
* second merge of Data Link with Allocations df

In [57]:
final_join[['Award_Year','Local_Agency','PPNO','Project_Title','TIRCP_project_sheet','tot_exp','Expended_Amt_project_sheet','from_joined_alloc_df_tot_exp', '_merge']]

,Award_Year,Local_Agency,PPNO,Project_Title,TIRCP_project_sheet,tot_exp,Expended_Amt_project_sheet,from_joined_alloc_df_tot_exp,_merge
0,2015,Antelope Valley Transit Authority (AVTA),CP005,Regional Transit Interconnectivity & Environme...,24403000.00,NaN,21714177.53,22284205.53,both
1,2015,Capitol Corridor Joint Powers Authority,CP012,Travel Time Reduction Project,4620000.00,NaN,4619999.90,4619999.90,both
2,2015,Los Angeles County Metropolitan Transportation...,CP015,Willowbrook/Rosa Parks Station & Blue Line Lig...,38494000.00,NaN,38494000.00,38494000.00,both
3,2015,Los Angeles-San Diego-San Luis Obispo Rail Cor...,CP007,Pacific Surfliner Transit Transfer Program,1675000.00,277840.47,277840.47,277840.47,both
4,2015,Montery-Salinas Transit,CP013,Monterey Bay Operations and Maintenance Facili...,10000000.00,NaN,0.00,10000000.00,both
5,2015,Orange County Transportation Authority (OCTA),CP004,Bravo! Route 560 Rapid Buses,2320000.00,2320000.00,0.00,2320000.00,both
6,2015,Sacramento Regional Transit District (SacRT),CP001,Sacramento Regional Transit's Refurbishment of...,6427000.00,NaN,0.00,6315208.89,both
7,2015,San Diego Association of Governments (SANDAG),CP003,South Bay Bus Rapid Transit,4000000.00,NaN,4000000.00,4000000.00,both
8,2015,San Diego Metropolitan Transit System (MTS),CP008,San Diego Metropolitan Transit System Trolley ...,31986000.00,NaN,28176000.00,31936000.00,both
9,2015,San Francisco Municipal Transportation Agency,CP006,SFMTA Light Rail Vehicle Fleet Expansion,41181000.00,41181000.00,0.00,41181000.00,both


In [55]:
#dropping cols I don't need
final_join2 = final_join[[
    'Award_Year', 'Project_#', 'Local_Agency', 'Vendor_ID_#',
       'Project_Title', 'District', 'County', 'Key_Project_Elements',
       'Master_Agreement_Number', 'Master_Agreement_Expiration_Date',
       'Project_Manager', 'Regional_Coordinator',
       'Technical_Assistance-CALTP_(Y/N)', 'Technical_Assistance-Fleet_(Y/N)',
       'Technical_Assistance-Network_Integration_(Y/N)',
       'Technical_Assistance-Priority_Population_(Y/N)', 'Total_Project_Cost',
       'TIRCP_project_sheet', 'Allocated_Amount',
       'Unallocated_amt_project_sheet', 'Percentge_Allocated',
       'Other_Funds_Involved', 'Award_Cycle',
       'Local_Agency_Address', 'Local_Agency_City', 'Local_Agency_Zip',
       'Local_Agency_Contact', 'Local_Agency_Email',
       'Local_Agency_Phone_Number', 'Comments/Additional_Contacts', 'PPNO',
       'from_joined_alloc_df_tot_exp']]

In [62]:
final_join2.dtypes

Award_Year                                                 int64
Project_#                                                  int64
Local_Agency                                              object
Vendor_ID_#                                               object
Project_Title                                             object
District                                                  object
County                                                    object
Key_Project_Elements                                      object
Master_Agreement_Number                                   object
Master_Agreement_Expiration_Date                  datetime64[ns]
Project_Manager                                           object
Regional_Coordinator                                      object
Technical_Assistance-CALTP_(Y/N)                          object
Technical_Assistance-Fleet_(Y/N)                          object
Technical_Assistance-Network_Integration_(Y/N)            object
Technical_Assistance-Prio

In [70]:
final_join2.isna().sum()

Award_Year                                         0
Project_#                                          0
Local_Agency                                       0
Vendor_ID_#                                        0
Project_Title                                      0
District                                           8
County                                             4
Key_Project_Elements                               0
Master_Agreement_Number                            7
Master_Agreement_Expiration_Date                   7
Project_Manager                                    0
Regional_Coordinator                               3
Technical_Assistance-CALTP_(Y/N)                  19
Technical_Assistance-Fleet_(Y/N)                  24
Technical_Assistance-Network_Integration_(Y/N)    22
Technical_Assistance-Priority_Population_(Y/N)    22
Total_Project_Cost                                 0
TIRCP_project_sheet                                0
Allocated_Amount                              

In [69]:
#subset final joined data frame for projects that sitll have N/A in expenditures
final_join2_subset = final_join2[['Award_Year','PPNO','Project_Title', 'from_joined_alloc_df_tot_exp']]

#### Still have 25 values missing :( 

In [69]:
final_join2_subset.loc[final_join2_subset['from_joined_alloc_df_tot_exp'].isnull()]

,Award_Year,PPNO,Project_Title,from_joined_alloc_df_tot_exp
18,2016,CP076,"Transforming California: Bus Electrification, ...",NaN
20,2016,CP058,Airport Metro Connector 96th Street Station/Me...,NaN
25,2016,CP080,Downtown/Riverfront Sacramento-West Sacramento...,NaN
31,2016,CP057,BART Silicon Valley Phase II Extension,NaN
32,2016,CP026,SB 132 ACE Extension Lathrop to Ceres/Merced,NaN
39,2018,CP029,Los Angeles City: Leading the Transformation t...,NaN
40,2018,CP054,Electric Blue: Electrification of City of Sant...,NaN
42,2018,CP030,Los Angeles Region Transit System Integration ...,NaN
48,2018,CP077,Ride Between the Line: Enhancing Access to Tra...,NaN
54,2018,CP078,SamTrans Express Bus Pilot,NaN
